In [1]:
from src.data import read_data,prepare_data
from src.models import train_model,predict_model
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
PATH_TO_TRAIN = '../data/raw/train15.csv'
PATH_TO_PREDICTORS = '../data/raw/predictors15.csv'

ONE_DAY = 24
ONE_WEEK = 7 * ONE_DAY
ONE_MONTH = 30* ONE_DAY

TRAIN_SIZE = 16 * ONE_MONTH
number_of_tests = ON
size_of_prediction = ONE_DAY
TEST_SIZE = number_of_tests * size_of_prediction

ROLLING_COLUMNS = ['POWER','VAR78', 'VAR79', 'VAR134', 'VAR157', 'VAR164', 'VAR165', 'VAR166', 'VAR167','VAR169', 'VAR175', 'VAR178', 'VAR228']
ACCUMLATED_FEATURE_COLUMNS = ['VAR169', 'VAR175', 'VAR178', 'VAR228']

df_original = read_data.read_data(PATH_TO_TRAIN,PATH_TO_PREDICTORS)

df_cut = df_original[:TRAIN_SIZE + TEST_SIZE]

intervals = []
for i in range(1,31):
    intervals.append(i*ONE_DAY)
for i in range(1,5):
    intervals.append(i*ONE_MONTH)

In [3]:
df = df_cut.copy()
df = prepare_data.prepare(df)
df = prepare_data.dissipate_features(df, ACCUMLATED_FEATURE_COLUMNS)
#df = prepare_data.add_rolling(df,["POWER"], intervals, ONE_DAY)
#ROLLING_COLUMNS.remove("POWER")
df = prepare_data.add_rolling(df,ROLLING_COLUMNS, intervals, ONE_DAY)

In [4]:
df_train = df[:TRAIN_SIZE].dropna(axis=0)
X_train = df_train.drop(ROLLING_COLUMNS,axis=1)
y_train = df_train.POWER
from xgboost import XGBRegressor
boosted_model = train_model.train(X_train, y_train)

In [5]:
import numpy as np
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
output_notebook()
from sklearn.metrics import explained_variance_score

results = []
y_tests = []
y_preds = []

for i in range(number_of_tests):
    start = TRAIN_SIZE+size_of_prediction * i
    end = TRAIN_SIZE+size_of_prediction * (i+1)
    df_test = df[start:end] 
    X_test = df_test.drop(ROLLING_COLUMNS,axis=1)
    y_test = df_test.POWER
    y_pred = predict_model.predict(X_test, boosted_model)
#     p = figure()
#     p.line(np.arange(len(y_test)), y_test, legend="real")
#     p.line(np.arange(len(y_pred)), y_pred, legend="predicted_xgb", color="orange")
#     show(p)
    y_tests.extend(y_test)
    y_preds.extend(y_pred)
    print(explained_variance_score(y_pred, y_test))
    results.append(explained_variance_score(y_pred, y_test))

Loading BokehJS ...

0.792711010328709
0.52786471127549
0.6480802034771165
0.8282803607006792
0.4966414264252651
0.3056702753311511
0.5350279927371123
0.8853575193404761
0.8527519545269002
0.6455253813753749
0.6841675884803309
0.6580588322213636
0.29923779122732297
0.8930802474355525
0.7959961281982147
0.33576875351612845
0.7385963743806335
0.4055018764311996
0.5324342498107759
0.7620098139382501
0.8831442799930036
0.7796767573710929
0.886993377667354
0.740214328250638
0.7534660722255393
0.791895935129395
0.7642716509303216
0.9103041083107526
0.8723146273255837
0.32809410130111794
0.8178940844290841
0.8800207127408305
0.8175000127771878
0.8672576025586252
0.6887848554493782
0.5235273107544617
0.5370696115671374
0.7673436139239413
0.9356095116519787
0.9293743714209065
0.9408675742532996
0.7626359312364995
0.8606625468513937
0.8047807160228367
0.7503122837621008
0.7321179367392294
0.9349717710071451
0.8188097702188614
0.7845901438551034
0.9491340609251893
0.8881025922797252
0.8390121510245739
0.7596482063283

0.6054585226492137
0.07421024684132183
0.4023060105740943
0.5911672749077452
0.5367878277217302
0.3688283168391032
0.177004548203757
0.8726293041137987
0.49433864326485766
0.499700146669121
0.2795194347243749
0.6452414077740242
0.909479355847413
0.5092897680362469
0.7496931870072017
0.34482610255926793
0.54480698075395
0.4498549799055692
0.3443085555327806
0.7434270703083774
0.26507800868271447
0.8733138291712046
0.7617408903508418
0.6430309083291939
0.5668441130666019
0.5079551753824069
0.3780652447056023
0.5246914218113219
0.512136502894085
0.5009799900404444
0.7613560975235673
0.6036673632518996
0.6004156276601633
0.5266384622743681
0.563180761635724
0.6905484746593461
0.37689937092242
0.6320585109013991
0.5033737672790071
0.49017948494452046
0.5004612258397936
0.49483556862576683
0.5266341148885267
0.6312203772245728
0.40813133472084817
0.5070326525738121
0.6552787373461922
0.49593752064296903
0.553412155852196
-0.032166190508901016
0.6184638379677365
0.662616737282699
0.6691357920

In [12]:
p = figure()
p.line(np.arange(7*ONE_DAY), y_tests[:7*ONE_DAY], legend="real", line_width=2)
p.line(np.arange(7*ONE_DAY), y_preds[:7*ONE_DAY], legend="predicted", color="orange", line_width=2)
show(p)

In [7]:
p = figure()
p.line(np.arange(len(results)), results, legend="accuracy", line_width=2,color = "green")
show(p)

In [8]:
# # -*- coding: utf-8 -*-
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.metrics import mean_absolute_error
# import matplotlib.pylab as plt
# import pandas as pd, numpy as np
# from sklearn.metrics import mean_absolute_error
# import xgboost as xgb
# from hyperopt import hp, tpe, STATUS_OK, Trials
# from hyperopt.fmin import fmin

# def xgb_eval_mae(yhat, dtrain):
#     y = dtrain.get_label()
#     return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

# def objective(space):

#     clf = xgb.XGBRegressor(n_estimators = space['n_estimators'],
#                            max_depth = space['max_depth'],
#                            min_child_weight = space['min_child_weight'],
#                            subsample = space['subsample'],
#                            learning_rate = space['learning_rate'],
#                            gamma = space['gamma'],
#                            colsample_bytree = space['colsample_bytree'],
#                            objective='reg:linear',
#                            nthread=8
#                            )

#     eval_set  = [( X_train, y_train), ( X_test, y_test)]

#     clf.fit(X_train,
#             y_train,
#             eval_set=eval_set,
#             eval_metric = 'mae')

#     pred = clf.predict(X_test)
# #   mae = mean_absolute_error(np.exp(y_valid), np.exp(pred))
#     mae = mean_absolute_error((y_test), (pred))

#     print("SCORE:", mae)
#     return{'loss':mae, 'status': STATUS_OK }


# space ={
#         'max_depth': hp.choice('max_depth', np.arange(10, 30, dtype=int)),
#         'min_child_weight': hp.quniform ('min_child', 1, 20, 1),
#         'subsample': hp.uniform ('subsample', 0.8, 1),
#         'n_estimators' : hp.choice('n_estimators', np.arange(1000, 10000, 100, dtype=int)),
#         'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
#         'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
#         'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05)
#     }

# # space ={
# #         'max_depth': hp.choice('max_depth', np.arange(10, 11, dtype=int)),
# #         'min_child_weight': hp.quniform ('min_child', 1,2,1),
# #         'subsample': hp.uniform ('subsample', 0.8,1),
# #         'n_estimators' : hp.choice('n_estimators', np.arange(100, 1000, 10, dtype=int)),
# #         'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
# #         'gamma' : hp.quniform('gamma', 0.5, 1, 0.5),
# #         'colsample_bytree' : hp.quniform('colsample_bytree', 0.5,1, 0.5)
# #        }


# trials = Trials()
# best = fmin(fn=objective,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=3, # change
#             trials=trials)

# print(best)

In [9]:
#{'colsample_bytree': 1.0, 'gamma': 1.0, 'learning_rate': 0.45, 'max_depth': 0, 'min_child': 2.0, 'n_estimators': 32, 'subsample': 0.9025912497131673}
#{'colsample_bytree': 0.9, 'gamma': 0.65, 'learning_rate': 0.5, 'max_depth': 1, 'min_child': 9.0, 'n_estimators': 45, 'subsample': 0.9474048013202294}